# GCC118 - Programação Matemática  
## Universidade Federal de Lavras  

---

## Problema 2  

### Alunos:  
- **Gabriel Jardim de Souza** - 202310530  
- **Paulo Henrique Silveira dos Anjos** - 202310533


## Enunciado:

Uma certa fábrica de camisetas deseja aproveitar as finais de um campeonato de futebol para vender camisetas dos times envolvidos. Os jogos vão durar quatro semanas. O custo de produção de cada camiseta é de 2,00 reais nas duas primeiras semanas e 2,50 reais nas duas últimas, quando a concorrência demandar por material no mercado. A demanda semanal de camisetas será de 5.000, 10.000, 30.000 e 60.000. A capacidade máxima de produção da empresa é de 25.000 camisetas semanalmente. Na primeira e na segunda semanas, a empresa poderá contratar horas extras de serviço e fabricar mais 10.000 camisetas em cada semana. Nesse caso, o custo de produção sobe para 2,80 reais. O excesso de produção pode ser estocado a um custo de R$ 0,20 por unidade por semana. Formule um modelo que minimize os custos.

---

### Objetivo

Formule um modelo que **minimize os custos**.

## Instalação da biblioteca PuLP

In [4]:
!pip install pulp
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

## Declaração dos parâmetros

* $T$: conjunto de semanas do planejamento (0, 1, 2, 3).

* $custo\_normal$: lista com o custo de produção normal (R$) em cada semana.

* $demanda\_normal$: lista com a demanda de camisetas (unidades) em cada semana.

* $demanda\_extra$: lista com a capacidade máxima de produção extra (unidades) em cada semana.

* $custo\_extra$: lista com o custo de produção extra (R$) em cada semana.

* $custo\_estoque$: lista com o custo de estocagem por unidade de camiseta estocada (R$) em cada semana.


In [5]:
custo_normal = [2.0, 2.0, 2.5, 2.5]

demanda_normal = [5000, 10000, 30000, 60000]

demanda_extra = [10000, 10000, 0, 0]

custo_extra = [2.80, 2.80, 0, 0]

custo_estoque = [0.2, 0.2, 0.2, 0.2]

T = range(4)

## Declaração do objeto que representa o modelo matemático

In [6]:
modelo = LpProblem("Venda_Camisetas", LpMinimize)

## Variáveis de decisão

* $x_t$: quantidade de camisetas produzidas em regime normal na semana $t$, com $t \in T$, $x_t \geq 0$.

* $y_t$: quantidade de camisetas produzidas em regime de horas extras na semana $t$, com $t \in T$, $y_t \geq 0$.

* $e_t$: quantidade de camisetas em estoque ao final da semana $t$, com $t \in T$, $e_t \geq 0$.


In [7]:
x = LpVariable.dicts('producao_normal', T, lowBound=0)
e = LpVariable.dicts('estoque', T, lowBound=0)
y = LpVariable.dicts("producao_extra", T, lowBound=0)

## Função Objetivo

* Minimização do custo total:  

$\min \sum_{t \in T} \Big( custo\_normal[t] \cdot x_t \;+\; custo\_extra[t] \cdot y_t \;+\; custo\_estoque[t] \cdot e_t \Big)$.



In [8]:
modelo += lpSum(custo_normal[t] * x[t] +
                custo_extra[t]*y[t] +
                custo_estoque[t]*e[t] for t in T)

## Restrições

* **Capacidade normal de produção (por semana):**  
$x_t \leq 25\,000 \quad \forall t \in T$

* **Capacidade de produção extra:**  
$y_t \leq 10\,000 \quad \text{para } t = 0,1$  
$y_t = 0 \quad \text{para } t = 2,3$

* **Balanço de estoque:**  

  Para a primeira semana (\(t=0\)):  
  $x_0 + y_0 - d_0 = e_0$
  
  Para as semanas seguintes (\(t = 1,2,3\)):  
  $e_{t-1} + x_t + y_t - d_t = e_t$

* **Não-negatividade:**  
$x_t \geq 0,\; y_t \geq 0,\; e_t \geq 0 \quad \forall t \in T$

In [9]:
for t in T:
    #capacidade normal
    modelo += x[t] <= 25000

    #capacidade extra semanas 1 e 2 (0 e 1 no array)
    if t in [0,1]:
        modelo += y[t] <= demanda_extra[t]
    else:
        modelo += y[t] == demanda_extra[t]

# Balanço de estoque
for t in T:
    if t == 0:
        modelo += x[t] + y[t] - demanda_normal[t] == e[t]
    else:
        modelo += e[t-1] + x[t] + y[t] - demanda_normal[t] == e[t]

## Resolvendo o problema

In [10]:
status = modelo.solve()

## Imprimindo as soluções do problema

In [11]:
status = modelo.solve()
print('status: ', LpStatus[status])
print('funcao objetivo: ', modelo.objective.value())
print('solucoes')
for t in T:
    print(f"Semana {t+1}: Normal={x[t].value()}, Extra={y[t].value()}, Estoque final={e[t].value()}")

status:  Optimal
funcao objetivo:  258000.0
solucoes
Semana 1: Normal=25000.0, Extra=0.0, Estoque final=20000.0
Semana 2: Normal=25000.0, Extra=5000.0, Estoque final=40000.0
Semana 3: Normal=25000.0, Extra=0.0, Estoque final=35000.0
Semana 4: Normal=25000.0, Extra=0.0, Estoque final=0.0
